## PROCESS DATA FILE FROM set-siamchart
Goal: to create an initial file for price data. Will create another job to update this file periodically from set data

In [1]:
%run 'header.ipynb'

In [171]:
import glob
path='/Users/sunny/Downloads/set-archive_EOD_UPDATE/'
all_files = glob.glob(path+'*.csv')

df=pd.DataFrame()
for fn in all_files:
    df = df.append(pd.read_csv(fn))
    
#set columns
df.columns = [x.replace('<','').replace('>','').lower() for x in df.columns]
df.rename(columns={'dtyyyymmdd':'date'},inplace=True)
df['date']=df['date'].astype('str')

#format dates
df['date']=pd.to_datetime(df['date'],format='%Y%m%d').dt.date
df.set_index('date',inplace=True)

#save dagta
ensure_dir('/Users/sunny/GoogleDrive/SunnyDoc/StockData/thai_stocks_price_data_initial.csv')
df.to_csv('/Users/sunny/GoogleDrive/SunnyDoc/StockData/thai_stocks_price_data_initial.csv')

In [2]:
%run 'header.ipynb'
df = pd.read_csv('/Users/sunny/GoogleDrive/SunnyDoc/StockData/thai_stocks_price_data_initial.csv',index_col=[0])
df.index = pd.DatetimeIndex(df.index).date
df.index.name='date'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from potzy.setdatareader import *
from metadata import *
all_tickers= get_all_thai_tickers()
len(all_tickers)

746

In [11]:
dff = df.copy()
bad_tickers=[]
for ticker in all_tickers:
    try:
        tmp=load_price_data_from_set(ticker)
        tmp.set_index('date',inplace=True)
        tmp.index = pd.to_datetime(tmp.index,dayfirst=True).date
        tmp.index.name='date'
        tmp.sort_index(inplace=True)
        tmp['ticker']=ticker

        #add volumne columns
        volumne_col='total volume(shares)'
        tmp['vol']=tmp[volumne_col]

        #filter columne
        cols=list(df.columns)
        tmp=tmp[cols]

        #filter date
        tmp=tmp[tmp.index>=df[df.ticker==ticker].index.max()]
        
        ## append
        dff=dff.append(tmp)
    except:
        bad_tickers.append(ticker)
        print("Bad ticker: ",ticker)

dff.drop_duplicates(inplace=True)

Bad ticker:  CPNRF
Bad ticker:  DTCPF
Bad ticker:  PRANDA
Bad ticker:  SSTSS
Bad ticker:  TFUND
Bad ticker:  TGROWTH
Bad ticker:  TLOGIS
Bad ticker:  UNIPF
Bad ticker:  WHAPF


In [34]:
#sanity check data
dff.to_csv('/Users/sunny/GoogleDrive/SunnyDoc/StockData/thai_stocks_price_data_siamchart_plus_set.csv')

## ADD SPLITTING DATA

In [1]:
%run 'header.ipynb'

In [2]:
## load 
df = pd.read_csv('/Users/sunny/GoogleDrive/SunnyDoc/StockData/thai_stocks_price_data_siamchart_plus_set.csv',index_col=[0])
df.index = pd.DatetimeIndex(df.index).date
df.index.name='date'

/Users/sunny/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#add previous date for debugging
#df.sort_index(inplace=True)
#df['close_prev']=df.groupby('ticker')['close'].transform(lambda x: x.shift(1))
#df['close_next']=df.groupby('ticker')['close'].transform(lambda x: x.shift(-1))

### load splitting data
df_split = pd.read_csv('../data/meta/thai_stock_splitting.csv',index_col=[0])
df_split['date']=pd.to_datetime(df_split['date'].astype('str'),format='%Y-%m-%d').dt.date
df_split.set_index('date',inplace=True)
df_split.sort_index(inplace=True)

## add cumulative adj_factor
df_split['adj_factor']=df_split['par']*1.0/df_split['par_prev']
df_split['cum_adj_factor']=df_split.groupby('ticker')['adj_factor'].transform(lambda x: np.cumprod(x[::-1])[::-1])

## merge 
print(len(df),len(df_split))
cols=['date','ticker','cum_adj_factor']
df=pd.merge(df.reset_index(),df_split.reset_index()[cols],how='outer',on=['date','ticker']).set_index('date')
print(len(df))

1329366 115
1329444


In [ ]:
all_tickers = list(df.ticker.unique())
df_all=pd.DataFrame()
for ticker in all_tickers:
    tmp=df[df.ticker==ticker].sort_index()
    tmp['adj_factor']=tmp['cum_adj_factor'].fillna(method='bfill').fillna(1.0).shift(-1)
    df_all = df_all.append(tmp)
    
# df.sort_index(inplace=True)
# tmp=df.groupby('ticker')['cum_adj_factor'].transform(lambda x: x.fillna(method='bfill').fillna(1.0).shift(-1))
# df['adj_factor']=tmp.values
# del tmp

In [ ]:
len(df_all)

In [ ]:
## adjust price
price_cols = ['open','high','low','close']
adj_price_cols = ['adj_'+x for x in price_cols]
for i in range(len(price_cols)):
    df_all[adj_price_cols[i]]=df_all[price_cols[i]]*df_all['adj_factor']
    
df_all=df_all[['ticker']+price_cols+adj_price_cols+['adj_factor']]

today_str = dt.datetime.today().date().strftime('%Y%m%d')
df_all.to_csv('../data/price/thai_price_all_'+today_str+'.csv')

/Users/sunny/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1329366 115
1329444
